In [4]:
import MeCab
import joblib
import numpy as np
from gensim.models.doc2vec import TaggedDocument
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
# MeCabの初期化
mecab = MeCab.Tagger("")

# サンプルの日本語文章
documents = [
    "これはテスト文章です。",
    "もう一つのサンプル文です。",
    "形態素解析を行い、Doc2Vecモデルを作成します。"
]

# 形態素解析を行う関数（特定の品詞を抽出）
def tokenize(text):
    node = mecab.parseToNode(text)
    tokens = []
    while node:
        feature = node.feature.split(',')
        if feature[0] in ['名詞', '動詞', '形容詞']:  # 名詞、動詞、形容詞のみ抽出
            tokens.append(node.surface)
        node = node.next
    return tokens

# ドキュメントを形態素解析し、タグ付きドキュメントとして格納
tagged_data = []
for i, doc in enumerate(documents):
    tokens = tokenize(doc)  # ドキュメントをトークン化
    tagged_document = TaggedDocument(words=tokens, tags=[str(i)])  # トークンとタグを用いてTaggedDocumentを作成
    tagged_data.append(tagged_document)  # tagged_dataリストに追加

# TF-IDF モデルの作成
corpus = [" ".join(td.words) for td in tagged_data]
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(corpus)

# インスタンスの保存
joblib.dump(vectorizer, "vectorizer.joblib")
joblib.dump(tfidf_matrix, "tfidf_matrix.joblib")

['tfidf_matrix.joblib']

In [6]:
# インスタンスのロード
vectorizer = joblib.load("vectorizer.joblib")
tfidf_matrix = joblib.load("tfidf_matrix.joblib").toarray()

# 新しいドキュメントの類似度を計算
new_document = "これは新しい文章です。"
new_vector = vectorizer.transform([" ".join(tokenize(new_document))]).toarray()[0]

# 既存のドキュメントと新しいドキュメントの類似度を計算
similarities = []
for i in range(len(documents)):
    doc_vector = tfidf_matrix[i]
    similarity = np.dot(new_vector, doc_vector) / (
                 np.linalg.norm(new_vector) * np.linalg.norm(doc_vector))
    similarities.append((i, similarity))

# 類似度で降順ソート
similarities.sort(key=lambda x: x[1], reverse=True)

# 類似度の結果を表示
for idx, similarity in similarities:
    print(f"Document {idx} Similarity: {similarity:.4f}")

Document 0 Similarity: 0.7071
Document 1 Similarity: 0.0000
Document 2 Similarity: 0.0000
